# **QUESTION 1**

In [ ]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.image import resize
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dir = "/content/drive/MyDrive/Lab4/Cat-dog-dataset"

In [ ]:
def data_preprocessing(folder_path, classes_folder, desired_shape = (224, 224)):
    images = []
    labels = []

    for i, label_name in enumerate(classes_folder):
        images_folder_path = os.path.join(folder_path, label_name)
        all_files = os.listdir(images_folder_path)
        for filename in all_files:
            if filename.endswith(('.png','.jpg', '.jpeg')):
                file_path = os.path.join(images_folder_path, filename)
                img = load_img(file_path, target_size = desired_shape)
                img_array = img_to_array(img)
                images.append(img_array)
                labels.append(i)
    return np.array(images), np.array(labels)

classes = ['cats', 'dogs']
data, labels = data_preprocessing(dir, classes)
labels = to_categorical(labels, num_classes = len(classes))
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2, random_state = 42)
X_train, X_test = X_train/255.0, X_test/255.0

In [ ]:
input_shape = (224, 224, 3)
input_layer = Input(shape = input_shape)

In [ ]:
filter_size = (3,3)

# Block 1
x = Conv2D(64, filter_size, activation = 'relu', padding = 'same')(input_layer)
x = Conv2D(64, filter_size, activation = 'relu', padding = 'same')(x)
x = MaxPooling2D((2,2), strides=(2,2))(x)

# Block 2
x = Conv2D(128, filter_size, activation = 'relu', padding = 'same')(x)
x = Conv2D(128, filter_size, activation = 'relu', padding = 'same')(x)
x = MaxPooling2D((2,2), strides=(2,2))(x)

# Block 3
x = Conv2D(256, filter_size, activation = 'relu', padding = 'same')(x)
x = Conv2D(256, filter_size, activation = 'relu', padding = 'same')(x)
x = Conv2D(256, filter_size, activation = 'relu', padding = 'same')(x)
x = Conv2D(256, filter_size, activation = 'relu', padding = 'same')(x)
x = MaxPooling2D((2,2), strides=(2,2))(x)

# Block 4
x = Conv2D(512, filter_size, activation = 'relu', padding = 'same')(x)
x = Conv2D(512, filter_size, activation = 'relu', padding = 'same')(x)
x = Conv2D(512, filter_size, activation = 'relu', padding = 'same')(x)
x = Conv2D(512, filter_size, activation = 'relu', padding = 'same')(x)
x = MaxPooling2D((2,2), strides=(2,2))(x)

# Block 5
x = Conv2D(512, filter_size, activation = 'relu', padding = 'same')(x)
x = Conv2D(512, filter_size, activation = 'relu', padding = 'same')(x)
x = Conv2D(512, filter_size, activation = 'relu', padding = 'same')(x)
x = Conv2D(512, filter_size, activation = 'relu', padding = 'same')(x)
x = MaxPooling2D((2,2), strides=(2,2))(x)

# Flatten
x = Flatten()(x)


# Fully connected layers
x = Dense(4096, activation = 'relu')(x)
x = Dense(4096, activation = 'relu')(x)

# Final Dense Layer
final_dense_layer = Dense(len(classes), activation = "softmax")(x)

In [ ]:
# creating a model
model = Model(
    inputs = input_layer,
    outputs = final_dense_layer
    )

In [ ]:
model.compile(optimizer = Adam(learning_rate = 0.001), loss = "categorical_crossentropy", metrics = ['accuracy'])

In [ ]:
# Model summary
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4096)           │    16,781,31

 Total params: 139,578,434 (532.45 MB)

 Trainable params: 139,578,434 (532.45 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#training the model
training = model.fit(X_train,y_train, epochs = 10, batch_size = 32,validation_data = (X_test, y_test))


Epoch 1/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5118 - loss: 1.9924   

In [ ]:
# accuracy
model_accuracy = model.evaluate(X_test, y_test, verbose = 0)

In [ ]:
# Savin Model
model.save("VGG_19.h5")

In [ ]:
plt.figure(figsize=(10, 6))
plt.title("Accuracy Plotting")
plt.plot(training.history['accuracy'], label = 'Accuracy')
plt.plot(training.history['val_accuracy'], label = "Validation Accuracy")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.show()

---
###**Role of each layer in the architecture**
*   **Convolution:** used for feature extraction
*   **MaxPooling2D:** Detects the necessary details from an image

*   **ReLU:** An activation function that adds non linearity and thus helps learning non-linear patterns

*   **Flatten layer:** Converts the 3D feature maps to a 1D vector as an input for the dense layers following it.
*   **Dense layer:** Makes a fully connected layer, connecting all the neurons on one layer with all other of the next layer.
*   **Softmax layer:** An activation function that returns class probabilities as output
---


---
###**Why VGG-19 uses small filters (3x3)**
=> VGG-19 uses small filters because it requires less number of parameters. Besides it helps capturing more details and thus extracting more features than the larger ones

---


# **QUESTION 2**

In [ ]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.image import resize
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.applications import ResNet50


In [ ]:
import kagglehub
path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")

Using Colab cache for faster access to the 'chest-xray-pneumonia' dataset.


***
The dataset selected is called Chest X-Ray Images (Pneumonia), which has 5856 files in total. Among which there are 5216 training images and 624 test images and 16 validation images. Besides it has two classes : PNEUMONIA & NORMAL.

***

In [ ]:
base_dir = "/kaggle/input/chest-xray-pneumonia/chest_xray"

In [ ]:
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

##**Data Preprocessing**

In [ ]:
def data_preprocessing(folder_path, classes_folder, desired_shape = (224, 224)):
    images = []
    labels = []

    for i, label_name in enumerate(classes_folder):
        images_folder_path = os.path.join(folder_path, label_name)
        all_files = os.listdir(images_folder_path)
        for filename in all_files:
            if filename.endswith(('.png','.jpg', '.jpeg')):
                file_path = os.path.join(images_folder_path, filename)
                img = load_img(file_path, target_size = desired_shape)
                img_array = img_to_array(img)
                images.append(img_array)
                labels.append(i)
    return np.array(images), np.array(labels)

classes = ['NORMAL', 'PNEUMONIA']
data, labels = data_preprocessing(train_dir, classes)
labels = to_categorical(labels, num_classes = len(classes))
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2, random_state = 42)
X_train, X_test = X_train/255.0, X_test/255.0

## **Using ResNet50**

In [ ]:
input_shape = X_train[0].shape

In [ ]:
base_resnet = ResNet50(weights = 'imagenet', include_top = False, input_shape=input_shape)
base_resnet.trainable = False
flatten_layer = Flatten()(base_resnet.output)
Dense_layer = Dense(64, activation = "relu")(flatten_layer)
final_layer = Dense(len(classes), activation = "softmax")(Dense_layer)

In [ ]:
model = Model(
    inputs = base_resnet.input,
    outputs = final_layer
    )

model.compile(optimizer = Adam(learning_rate=0.001),
              loss = "categorical_crossentropy", metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
resnet50 = model.fit(X_train, y_train, epochs = 10, batch_size = 32, validation_data = (X_test, y_test))

Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 43s 213ms/step - accuracy: 0.7090 - loss: 1.0242 - val_accuracy: 0.7251 - val_loss: 0.6508
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 14s 106ms/step - accuracy: 0.7455 - loss: 0.6372 - val_accuracy: 0.7251 - val_loss: 0.6215
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 14s 108ms/step - accuracy: 0.7494 - loss: 0.6052 - val_accuracy: 0.7251 - val_loss: 0.6040
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 14s 110ms/step - accuracy: 0.7479 - loss: 0.5875 - val_accuracy: 0.7251 - val_loss: 0.5949
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 14s 109ms/step - accuracy: 0.7531 - loss: 0.5728 - val_accuracy: 0.7251 - val_loss: 0.5905
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 14s 107ms/step - accuracy: 0.7357 - loss: 0.5809 - val_accuracy: 0.7251 - val_loss: 0.5886
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 14s 107ms/step - accuracy: 0.7514 - loss: 0.5644 - val_accuracy: 0.7251 - val_loss: 0.5881
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 20s 107ms/step - accuracy: 0.7449 - loss: 0

In [ ]:
# accuracy
model_accuracy_resnet50 = model.evaluate(X_test, y_test, verbose = 0)

In [ ]:
print("Test accuracy:",model_accuracy_resnet50[1])

Test accuracy: 0.725095808506012


In [ ]:
y_predicted_probs = model.predict(X_test)

33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 86ms/step


In [ ]:
y_predicted_val = np.argmax(y_predicted_probs, axis = 1)

In [ ]:
y_expected = np.argmax(y_test, axis = 1)

In [ ]:
# Classification Report
from sklearn.metrics import classification_report

print(classification_report(y_expected, y_predicted_val, target_names = classes))

              precision    recall  f1-score   support

      NORMAL       0.00      0.00      0.00       287
   PNEUMONIA       0.73      1.00      0.84       757

    accuracy                           0.73      1044
   macro avg       0.36      0.50      0.42      1044
weighted avg       0.53      0.73      0.61      1044



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


##**Using VGG-19**

In [ ]:
from tensorflow.keras.applications import VGG19


In [ ]:
input_shape = X_train[0].shape

In [ ]:
base_Vgg19 = VGG19(weights = 'imagenet', include_top = False, input_shape=input_shape)
base_Vgg19.trainable = False
flatten_layer = Flatten()(base_Vgg19.output)
Dense_layer = Dense(64, activation = "relu")(flatten_layer)
final_layer = Dense(len(classes), activation = "softmax")(Dense_layer)

In [ ]:
model_vgg = Model(
    inputs = base_Vgg19.input,
    outputs = final_layer
    )

model_vgg.compile(optimizer = Adam(learning_rate=0.001),
              loss = "categorical_crossentropy", metrics = ['accuracy'])

In [ ]:
model_vgg.summary()

In [ ]:
Vgg19 = model_vgg.fit(X_train, y_train, epochs = 10, batch_size = 32, validation_data = (X_test, y_test))

Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 71s 415ms/step - accuracy: 0.8390 - loss: 0.5280 - val_accuracy: 0.9722 - val_loss: 0.0829
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 36s 278ms/step - accuracy: 0.9720 - loss: 0.0702 - val_accuracy: 0.9789 - val_loss: 0.0649
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 35s 271ms/step - accuracy: 0.9830 - loss: 0.0503 - val_accuracy: 0.9789 - val_loss: 0.0579
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 36s 277ms/step - accuracy: 0.9840 - loss: 0.0421 - val_accuracy: 0.9808 - val_loss: 0.0527
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 36s 275ms/step - accuracy: 0.9899 - loss: 0.0283 - val_accuracy: 0.9856 - val_loss: 0.0455
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 36s 274ms/step - accuracy: 0.9941 - loss: 0.0205 - val_accuracy: 0.9722 - val_loss: 0.0686
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 36s 274ms/step - accuracy: 0.9921 - loss: 0.0241 - val_accuracy: 0.9866 - val_loss: 0.0437
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 36s 275ms/step - accuracy: 0.9974 - loss: 0

In [ ]:
# accuracy
model_accuracy_vgg19 = model_vgg.evaluate(X_test, y_test, verbose = 0)

In [ ]:
print("Test accuracy:",model_accuracy_vgg19[1])

Test accuracy: 0.9856321811676025


In [ ]:
y_predicted_probs = model_vgg.predict(X_test)

33/33 ━━━━━━━━━━━━━━━━━━━━ 9s 243ms/step


In [ ]:
y_predicted_val = np.argmax(y_predicted_probs, axis = 1)

In [ ]:
y_expected = np.argmax(y_test, axis = 1)

In [ ]:
# Classification Report
from sklearn.metrics import classification_report

print(classification_report(y_expected, y_predicted_val, target_names = classes))

              precision    recall  f1-score   support

      NORMAL       0.95      1.00      0.97       287
   PNEUMONIA       1.00      0.98      0.99       757

    accuracy                           0.99      1044
   macro avg       0.98      0.99      0.98      1044
weighted avg       0.99      0.99      0.99      1044



##**Using InceptionV3**

In [ ]:
from tensorflow.keras.applications import InceptionV3


In [ ]:
input_shape = X_train[0].shape

In [ ]:
base_InceptionV3 = InceptionV3(weights = 'imagenet', include_top = False, input_shape=input_shape)
base_InceptionV3.trainable = False
flatten_layer = Flatten()(base_InceptionV3.output)
Dense_layer = Dense(64, activation = "relu")(flatten_layer)
final_layer = Dense(len(classes), activation = "softmax")(Dense_layer)

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
model_InceptionV3 = Model(
    inputs = base_InceptionV3.input,
    outputs = final_layer
    )

model_InceptionV3.compile(optimizer = Adam(learning_rate=0.001),
              loss = "categorical_crossentropy", metrics = ['accuracy'])

In [ ]:
model_InceptionV3.summary()

In [ ]:
inception = model_InceptionV3.fit(X_train, y_train, epochs = 10, batch_size = 32, validation_data = (X_test, y_test))

Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 50s 242ms/step - accuracy: 0.8377 - loss: 2.9536 - val_accuracy: 0.9531 - val_loss: 0.1950
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 10s 79ms/step - accuracy: 0.9682 - loss: 0.1071 - val_accuracy: 0.9492 - val_loss: 0.1606
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 10s 80ms/step - accuracy: 0.9883 - loss: 0.0384 - val_accuracy: 0.9502 - val_loss: 0.1693
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 10s 79ms/step - accuracy: 0.9911 - loss: 0.0252 - val_accuracy: 0.9713 - val_loss: 0.1375
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 11s 83ms/step - accuracy: 0.9786 - loss: 0.0501 - val_accuracy: 0.9665 - val_loss: 0.1589
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 10s 80ms/step - accuracy: 0.9894 - loss: 0.0285 - val_accuracy: 0.9646 - val_loss: 0.1939
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 21s 82ms/step - accuracy: 0.9889 - loss: 0.0331 - val_accuracy: 0.9693 - val_loss: 0.1627
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 21s 83ms/step - accuracy: 0.9967 - loss: 0.0082 -

In [ ]:
# accuracy
model_accuracy_InceptionV3 = model_InceptionV3.evaluate(X_test, y_test, verbose = 0)

In [ ]:
print("Test accuracy:",model_accuracy_InceptionV3[1])

Test accuracy: 0.975095808506012


In [ ]:
y_predicted_probs = model_InceptionV3.predict(X_test)

33/33 ━━━━━━━━━━━━━━━━━━━━ 15s 285ms/step


In [ ]:
y_predicted_val = np.argmax(y_predicted_probs, axis = 1)

In [ ]:
y_expected = np.argmax(y_test, axis = 1)

In [ ]:
# Classification Report
from sklearn.metrics import classification_report

print(classification_report(y_expected, y_predicted_val, target_names = classes))

              precision    recall  f1-score   support

      NORMAL       0.99      0.92      0.95       287
   PNEUMONIA       0.97      0.99      0.98       757

    accuracy                           0.98      1044
   macro avg       0.98      0.96      0.97      1044
weighted avg       0.98      0.98      0.97      1044



----
###**Comparison**
For the selected dataset, VGG19 gives the best performance with an accuracy of 0.99. Becasue it handles class imbalance better. This is followed by InceptionV3 model, which has an accuracy of 0.98. For both classes it was able to handle the imbalance and give a balanced prediction. Because of being unable to do similarly, the worst performance so far is from the ResNet50 model, with an accuracy of 0.73.

----